# Data Consolidation & Cleaning

In [39]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## TransferMarkt Data Consolidation

In [37]:
########################## PLAYERS DF #########################
# Importing players dataframe 
players_df = pd.read_csv('/Users/enzovillafuerte/Desktop/Grad School/Data Science/Final Project/TransferMarkt Data/players.csv')

# keeping data only from last 5 seasons
years_intended = [2022, 2021, 2020, 2019, 2018]

# keeping players that played in last years or are currently playing 
players_df = players_df[players_df['last_season'].isin(years_intended)]

# keeping inteested variables
players_df = players_df[['player_id', 'name', 'last_season',
       'current_club_id', 'player_code', 'position', 'sub_position',
       'foot',
       'image_url', 'current_club_domestic_competition_id',
       'current_club_name']]

final_df = players_df.copy()


########################## CLUB DF ##############################
clubs_df = pd.read_csv('/Users/enzovillafuerte/Desktop/Grad School/Data Science/Final Project/TransferMarkt Data/clubs.csv')

clubs_df = clubs_df[['club_id', 'club_code', 'name', 'domestic_competition_id']]

# merging based on club_id
final_df = final_df.merge(clubs_df, left_on='current_club_id', right_on='club_id')


########################## COMPETITIONS DF ##############################
# here we are just interested in the country for easier cleaning
competitions_df = pd.read_csv('/Users/enzovillafuerte/Desktop/Grad School/Data Science/Final Project/TransferMarkt Data/competitions.csv')

competitions_df = competitions_df[['competition_id', 'type', 'country_name']]

# merging on competition_id
final_df = final_df.merge(competitions_df, left_on='current_club_domestic_competition_id', right_on='competition_id')


########################## PLAYER VALUATIONS DF ##############################
valuations_df = pd.read_csv('/Users/enzovillafuerte/Desktop/Grad School/Data Science/Final Project/TransferMarkt Data/player_valuations.csv')

# check for duplicates. Expecting that!
duplicates = valuations_df.duplicated(subset=['player_id'], keep=False)

# now we have to merge on this one to avoid data deletion. turns out final_df wasn't the final df xd
valuations_df = valuations_df.merge(final_df, left_on='player_id', right_on='player_id')


valuations_df

,player_id,last_season_x,datetime,date,dateweek,market_value_in_eur,n,current_club_id_x,player_club_domestic_competition_id,name_x,...,image_url,current_club_domestic_competition_id,current_club_name,club_id,club_code,name_y,domestic_competition_id,competition_id,type,country_name
0,532,2019,10/4/04 0:00,10/4/04,10/4/04,9500000,1,86,L1,Claudio Pizarro,...,https://img.a.transfermarkt.technology/portrai...,L1,SV Werder Bremen,86,sv-werder-bremen,SV Werder Bremen,L1,L1,domestic_league,Germany
1,532,2019,7/14/05 0:00,7/14/05,7/11/05,12000000,1,86,L1,Claudio Pizarro,...,https://img.a.transfermarkt.technology/portrai...,L1,SV Werder Bremen,86,sv-werder-bremen,SV Werder Bremen,L1,L1,domestic_league,Germany
2,532,2019,9/30/05 0:00,9/30/05,9/26/05,10000000,1,86,L1,Claudio Pizarro,...,https://img.a.transfermarkt.technology/portrai...,L1,SV Werder Bremen,86,sv-werder-bremen,SV Werder Bremen,L1,L1,domestic_league,Germany
3,532,2019,1/9/06 0:00,1/9/06,1/9/06,12000000,1,86,L1,Claudio Pizarro,...,https://img.a.transfermarkt.technology/portrai...,L1,SV Werder Bremen,86,sv-werder-bremen,SV Werder Bremen,L1,L1,domestic_league,Germany
4,532,2019,6/2/06 0:00,6/2/06,5/29/06,10500000,1,86,L1,Claudio Pizarro,...,https://img.a.transfermarkt.technology/portrai...,L1,SV Werder Bremen,86,sv-werder-bremen,SV Werder Bremen,L1,L1,domestic_league,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177063,894415,2022,7/4/23 0:00,7/4/23,7/3/23,175000,1,173,DK1,Tobias Augustinus-Jensen,...,https://img.a.transfermarkt.technology/portrai...,DK1,Odense Boldklub,173,odense-boldklub,Odense Boldklub,DK1,DK1,domestic_league,Denmark
177064,973311,2022,7/5/23 0:00,7/5/23,7/3/23,50000,1,3840,TR1,Hijran Ali Boyaci,...,https://img.a.transfermarkt.technology/portrai...,TR1,Adana Demirspor,3840,adana-demirspor,Adana Demirspor,TR1,TR1,domestic_league,Turkey
177065,717392,2022,7/21/23 0:00,7/21/23,7/17/23,50000,1,660,UKR1,Anton Glushchenko,...,https://img.a.transfermarkt.technology/portrai...,UKR1,Shakhtar Donetsk,660,shakhtar-donetsk,Shakhtar Donetsk,UKR1,UKR1,domestic_league,Ukraine
177066,969971,2023,8/12/23 0:00,8/12/23,8/7/23,50000,1,868,TR1,Arda Kumru,...,https://img.a.transfermarkt.technology/portrai...,TR1,MKE Ankaragücü,868,mke-ankaragucu,MKE Ankaragücü,TR1,TR1,domestic_league,Turkey


### Further Data Cleaning

In [49]:
########################## keep only those within top 5 leagues
countries = ['Spain', 'England', 'Germany', 'Italy', 'France', 'Netherlands']

valuations_df = valuations_df[valuations_df['country_name'].isin(countries)]


########################## domestic league?
valuations_df['type'].value_counts() # all are domestic leagues, so GOOD

########################## keeping only attackers and midfielders
positions_of_interest = ['Attack', 'Midfield']
valuations_df = valuations_df[valuations_df['position'].isin(positions_of_interest)]

########################## datetime to adjust to initial marketvalue (or maybe ending?) - the one that makes more sense\\\
valuations_df['date'] = pd.to_datetime(valuations_df['date'])

# Get records only from June (month 6 .- during the summer transfer window)
valuations_df = valuations_df[valuations_df['date'].dt.month == 6]

# Get records for last years only
valuations_df = valuations_df[(valuations_df['date'].dt.year == 2018) |
                              (valuations_df['date'].dt.year == 2019) |
                              (valuations_df['date'].dt.year == 2020) |
                              (valuations_df['date'].dt.year == 2021) |
                              (valuations_df['date'].dt.year == 2022)]

# keep only the year
valuations_df['date'] = valuations_df['date'].dt.year

########################## CONSIDER LIMITING TO LOWER RANKED LEAGUES FOR IMPROVED ACCURACY. NETHERLANDS & BUNDESLIGA FOR EXAMPLE. WOULD THAT BE ENOUGH DATA?


########################## Limiting players for lower than 30 years (MAYBE) ~ NO NEEDED WILL BE INCLUDED AS FEATURE ON FBREF DATA


########################## Keep desired columns
valuations_df = valuations_df[['player_id', 'last_season_x', 'date',
       'market_value_in_eur', 'current_club_id_x',
       'player_club_domestic_competition_id', 'name_x',
       'current_club_id_y', 'player_code', 'position', 'sub_position', 'foot',
       'image_url', 'current_club_domestic_competition_id',
       'current_club_name', 'club_id', 'club_code', 'name_y',
       'domestic_competition_id', 'competition_id', 'type', 'country_name']]


valuations_df

,player_id,last_season_x,date,market_value_in_eur,current_club_id_x,player_club_domestic_competition_id,name_x,current_club_id_y,player_code,position,...,image_url,current_club_domestic_competition_id,current_club_name,club_id,club_code,name_y,domestic_competition_id,competition_id,type,country_name
30,532,2019,2018,500000,86,L1,Claudio Pizarro,86,claudio-pizarro,Attack,...,https://img.a.transfermarkt.technology/portrai...,L1,SV Werder Bremen,86,sv-werder-bremen,SV Werder Bremen,L1,L1,domestic_league,Germany
32,532,2019,2019,500000,86,L1,Claudio Pizarro,86,claudio-pizarro,Attack,...,https://img.a.transfermarkt.technology/portrai...,L1,SV Werder Bremen,86,sv-werder-bremen,SV Werder Bremen,L1,L1,domestic_league,Germany
350,3455,2022,2021,4000000,5,IT1,Zlatan Ibrahimović,5,zlatan-ibrahimović,Attack,...,https://img.a.transfermarkt.technology/portrai...,IT1,AC Milan,5,ac-mailand,AC Milan,IT1,IT1,domestic_league,Italy
352,3455,2022,2022,3000000,5,IT1,Zlatan Ibrahimović,5,zlatan-ibrahimović,Attack,...,https://img.a.transfermarkt.technology/portrai...,IT1,AC Milan,5,ac-mailand,AC Milan,IT1,IT1,domestic_league,Italy
559,4018,2019,2018,1000000,167,L1,Daniel Baier,167,daniel-baier,Midfield,...,https://img.a.transfermarkt.technology/portrai...,L1,FC Augsburg,167,fc-augsburg,FC Augsburg,L1,L1,domestic_league,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175850,736034,2021,2022,200000,1390,IT1,Isaías Delpupo,1390,isaias-delpupo,Midfield,...,https://img.a.transfermarkt.technology/portrai...,IT1,Cagliari Calcio,1390,cagliari-calcio,Cagliari Calcio,IT1,IT1,domestic_league,Italy
175852,745436,2021,2022,100000,6574,IT1,Salim Abubakar,6574,salim-abubakar,Midfield,...,https://img.a.transfermarkt.technology/portrai...,IT1,US Sassuolo,6574,us-sassuolo,US Sassuolo,IT1,IT1,domestic_league,Italy
175857,857964,2022,2022,150000,800,IT1,Mannah Chiwisa,800,mannah-chiwisa,Midfield,...,https://img.a.transfermarkt.technology/portrai...,IT1,Atalanta BC,800,atalanta-bergamo,Atalanta BC,IT1,IT1,domestic_league,Italy
175864,988862,2021,2022,1000000,800,IT1,Moustapha Cissé,800,moustapha-cisse,Attack,...,https://img.a.transfermarkt.technology/portrai...,IT1,Atalanta BC,800,atalanta-bergamo,Atalanta BC,IT1,IT1,domestic_league,Italy


## Merging FBREF data

In [59]:
fbref_2018 = pd.read_csv('/Users/enzovillafuerte/Desktop/Grad School/Data Science/Final Project/FBREF Data/Final FBRef 2017-2018.csv')
fbref_2018['Season'] = 2018

fbref_2019 = pd.read_csv('/Users/enzovillafuerte/Desktop/Grad School/Data Science/Final Project/FBREF Data/Final FBRef 2018-2019.csv')
fbref_2019['Season'] = 2019

fbref_2020 = pd.read_csv('/Users/enzovillafuerte/Desktop/Grad School/Data Science/Final Project/FBREF Data/Final FBRef 2019-2020.csv')
fbref_2020['Season'] = 2020

fbref_2021 = pd.read_csv('/Users/enzovillafuerte/Desktop/Grad School/Data Science/Final Project/FBREF Data/Final FBRef 2020-2021.csv')
fbref_2021['Season'] = 2021

fbref_2022 = pd.read_csv('/Users/enzovillafuerte/Desktop/Grad School/Data Science/Final Project/FBREF Data/Final FBRef 2021-2022.csv')
fbref_2022['Season'] = 2022

# concatenating frames
frames = [fbref_2018, fbref_2019, fbref_2020, fbref_2021, fbref_2022]
total_fbref_df = pd.concat(frames)


total_fbref_df = total_fbref_df.merge(valuations_df, 
                              left_on=['Player', 'Season'],
                              right_on=['name_x', 'date']
                             )


In [61]:
total_fbref_df.to_csv('final_df.csv')